In [1]:
import nltk
# nltk.download('punkt')
# nltk.download('wordnet')
import json
import numpy as np
import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import random
from sklearn.preprocessing import LabelEncoder

C:\Users\n_int\AppData\Local\Temp\ipykernel_21856\3059552380.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
with open('intents.json') as file:
    data = json.load(file)

In [3]:
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [4]:
encoder = LabelEncoder()
training_labels  = encoder.fit_transform(training_labels)

In [5]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D

In [7]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [34]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim ,input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [35]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (  (None, 16)                0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 3)                 51        
                                                                 
Total params: 16595 (64.82 KB)
Trainable params: 16595 (64.82 KB)
Non-trainable params: 0 (0.00 Byte)
____________________

In [36]:
epochs = 550
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/550
1/1 [==============================] - 1s 746ms/step - loss: 1.0986 - accuracy: 0.2941
Epoch 2/550
1/1 [==============================] - 0s 3ms/step - loss: 1.0981 - accuracy: 0.3529
Epoch 3/550
1/1 [==============================] - 0s 2ms/step - loss: 1.0975 - accuracy: 0.4118
Epoch 4/550
1/1 [==============================] - 0s 4ms/step - loss: 1.0968 - accuracy: 0.4118
Epoch 5/550
1/1 [==============================] - 0s 2ms/step - loss: 1.0963 - accuracy: 0.4118
Epoch 6/550
1/1 [==============================] - 0s 3ms/step - loss: 1.0957 - accuracy: 0.4118
Epoch 7/550
1/1 [==============================] - 0s 3ms/step - loss: 1.0952 - accuracy: 0.4118
Epoch 8/550
1/1 [==============================] - 0s 3ms/step - loss: 1.0947 - accuracy: 0.4118
Epoch 9/550
1/1 [==============================] - 0s 3ms/step - loss: 1.0942 - accuracy: 0.4118
Epoch 10/550
1/1 [==============================] - 0s 3ms/step - loss: 1.0937 - accuracy: 0.4118
Epoch 11/550
1/1 [=========

In [38]:
# saving model
model.save("chat_model")

import pickle

# saving tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

# saving label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: chat_model\assets


INFO:tensorflow:Assets written to: chat_model\assets
